# Advanced Tool Calling

In [1]:
import langchain_core.tools
import pydantic

import sys
sys.path.append('../src/')

import simplechatbot
from simplechatbot.openai_chatbot import OpenAIChatBot

In [2]:
def example_tool_factory(chatbot: simplechatbot.ChatBot) -> list[langchain_core.tools.BaseTool]:
    class StoryOutline(pydantic.BaseModel):
        """Outline of the story."""
        title: str = pydantic.Field(description="Title of the story.")
        story_topic: str = pydantic.Field(description="The topic of the story.")
        narrative_arc: str = pydantic.Field(description="The narrative arc of the story.")

        ch1_title: str = pydantic.Field(description="Title of chapter 1 of the story.")
        ch1_description: str = pydantic.Field(description="Longer description of chapter 1.")

        ch2_title: str = pydantic.Field(description="Title of chapter 2 of the story.")
        ch2_description: str = pydantic.Field(description="Longer description of chapter 2.")

        ch3_title: str = pydantic.Field(description="Title of chapter 3 of the story.")
        ch3_description: str = pydantic.Field(description="Longer description of chapter 3.")

        def as_str(self) -> str:
            return f'title: {self.title}\n\ntopic: {self.story_topic}\n\nnarrative: {self.narrative_arc}\n\n(ch. 1) {self.ch1_title}: {self.ch1_description}\n\n(ch. 2) {self.ch2_title}: {self.ch2_description}\n\n(ch. 3) {self.ch3_title}: {self.ch3_description}'


    system_prompt = """
    You are designed to produce an outline of the story given some basic information from the user. 
    The outline includes a title, story topic, and narrative arc, as well as titles and descriptions 
    of three chapters. You are to take creative liberties in generating the outline. Each chapter should
    have its own narrative arc that contributes to the overall story arc. The plot should show character 
    growth and reflect very Progressive American values.
    """

    outline_structbot = chatbot.structbot_from_model(output_structure=StoryOutline, system_prompt=system_prompt)

    @langchain_core.tools.tool
    def story_outline_tool(description_of_story: str) -> StoryOutline:
        '''Create an outline of a story given a description of the story.'''
        return outline_structbot.invoke(description_of_story)

    return [story_outline_tool]

In [3]:
keychain = simplechatbot.APIKeyChain.from_json_file('../keys.json')

system_prompt = """
You are a regular chatbot with the capability to generate a story outline if the user requests.
ONLY do this if the user requests though.
"""

chatbot = OpenAIChatBot.new(
    model_name = 'gpt-4o-mini', 
    api_key=keychain['openai'],
    system_prompt=system_prompt,
    tool_factories=[example_tool_factory],
)

stream = chatbot.stream(f'Create an outline for a story about two friends that have a lot of fun together.')
result = stream.print_and_collect()
result.tool_calls[0]

ToolCallInfo(id='call_chgc1HpKmwPI9FFaqhAgbHI6', name='story_outline_tool', type='tool_call', args={'description_of_story': 'Two lifelong friends, Alex and Jamie, embark on a series of adventurous outings in their hometown. They explore local parks, visit quirky cafés, and attend vibrant street festivals. Throughout their journey, they encounter humorous mishaps and unexpected challenges that test their friendship but ultimately strengthen it. As they share laughs and support each other, they learn valuable life lessons about friendship, trust, and living in the moment. The story culminates in a heartfelt celebration of their bond, showcasing the joy of friendship and the memories created along the way.'}, tool=StructuredTool(name='story_outline_tool', description='Create an outline of a story given a description of the story.', args_schema=<class 'langchain_core.utils.pydantic.story_outline_tool'>, func=<function example_tool_factory.<locals>.story_outline_tool at 0x10de8f7e0>))

In [4]:
outline = result.execute_tools()['story_outline_tool'].return_value
print(outline.as_str())

title: Cherished Adventures

topic: The enduring power of friendship as experienced through local adventures.

narrative: Two best friends, Alex and Jamie, navigate the ups and downs of life through a series of adventurous outings in their hometown. Each outing is fraught with humorous mishaps and challenges that ultimately reinforce their bond and help them grow as individuals. The story showcases their journey of friendship, highlighting the importance of trust, support, and enjoying the moment, culminating in a celebration that honors their deep connection.

(ch. 1) Unexpected Delights: Alex and Jamie kick off their series of adventures by exploring a quirky local park known for its bizarre sculptures and themed gardens. As they navigate the whimsical paths, they encounter strange but friendly locals and participate in an impromptu art contest. A mishap involving an overzealous paint-splash leads to laughter and a messy realization about the beauty of imperfection in art and life. T